In [1]:
from poker_project.data import prepare

In [2]:
import pandas as pd
import glob
import os
from io import StringIO


In [3]:
# Prepare the data set, return a dictionary of hands, player history, and players in games.
hdb = prepare.prepare_holdem()


In [4]:
# For example, get players from the game w/ timestamp '802704132'
hdb['roster']['802704132']

['2', 'ShoelessJ', 'tvp']

In [5]:
# View the history of 'ShoelessJ' from that game.
hdb['players']['ShoelessJ'].loc['802704132']

players dealt          2
player position        2
bet preflop            B
bet flop               -
bet turn               -
bet river              -
bank start         23921
action               100
amount won           150
card_1              None
card 2              None
Name: 802704132, dtype: object

In [6]:
# Return the hand information for that game.
print(hdb['hands'].query("timestamp == '802704132'"))

   timestamp  game_num  hand_num  num_players_dealt  num_players_flop  \
5  802704132         2         6                  2                 0   

   flop_pot  num_players_turn  turn_pot  num_players_river  river_pot  \
5         0                 0         0                  0          0   

   num_players_showdown  showdown_pot card_1 card_2 card_3 card_4 card_5  
5                     1           150   None   None   None   None   None  


In [11]:
# Structure of hand history data frame
hdb['hands'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9447 entries, 0 to 9446
Data columns (total 17 columns):
timestamp               9447 non-null int64
game_num                9447 non-null int64
hand_num                9447 non-null int64
num_players_dealt       9447 non-null int64
num_players_flop        9447 non-null int64
flop_pot                9447 non-null int64
num_players_turn        9447 non-null int64
turn_pot                9447 non-null int64
num_players_river       9447 non-null int64
river_pot               9447 non-null int64
num_players_showdown    9447 non-null int64
showdown_pot            9447 non-null int64
card_1                  3371 non-null object
card_2                  3371 non-null object
card_3                  3371 non-null object
card_4                  1776 non-null object
card_5                  1157 non-null object
dtypes: int64(12), object(5)
memory usage: 1.2+ MB


In [8]:
# Structure of player history dataframe
hdb['players']['ShoelessJ'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 94 entries, 802704118 to 802706246
Data columns (total 11 columns):
players dealt      94 non-null object
player position    94 non-null object
bet preflop        94 non-null object
bet flop           94 non-null object
bet turn           94 non-null object
bet river          94 non-null object
bank start         94 non-null object
action             94 non-null object
amount won         94 non-null object
card_1             14 non-null object
card 2             14 non-null object
dtypes: object(11)
memory usage: 11.3+ KB


In [10]:
# Structure of roster dictionary
hdb['roster']

{'802072665': ['2', 'A8', 'jzk'],
 '802587475': ['3', 'greg', 'jzk', 'sagerbot^'],
 '802587556': ['2', 'greg', 'jzk'],
 '802587590': ['2', 'greg', 'jzk'],
 '802704118': ['2', 'ShoelessJ', 'tvp'],
 '802704132': ['2', 'ShoelessJ', 'tvp'],
 '802704137': ['2', 'ShoelessJ', 'tvp'],
 '802704152': ['2', 'ShoelessJ', 'tvp'],
 '802704201': ['2', 'ShoelessJ', 'tvp'],
 '802704250': ['2', 'ShoelessJ', 'tvp'],
 '802704256': ['2', 'ShoelessJ', 'tvp'],
 '802704266': ['2', 'ShoelessJ', 'tvp'],
 '802704268': ['2', 'ShoelessJ', 'tvp'],
 '802704284': ['2', 'ShoelessJ', 'tvp'],
 '802704287': ['2', 'ShoelessJ', 'tvp'],
 '802704298': ['2', 'ShoelessJ', 'tvp'],
 '802704346': ['2', 'ShoelessJ', 'tvp'],
 '802704382': ['2', 'ShoelessJ', 'tvp'],
 '802704385': ['2', 'ShoelessJ', 'tvp'],
 '802704410': ['2', 'ShoelessJ', 'tvp'],
 '802704545': ['2', 'ShoelessJ', 'tvp'],
 '802704549': ['2', 'ShoelessJ', 'tvp'],
 '802704578': ['2', 'ShoelessJ', 'tvp'],
 '802704627': ['2', 'ShoelessJ', 'tvp'],
 '802704630': ['2', 'Shoe